In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

In [ ]:
ratings_cols = ["user_id", "item_id", "rating", "timestamp"]
df_ratings = pd.read_csv(
    "../data/u.data", sep="\t", names=ratings_cols, encoding="latin-1"
)

In [ ]:
print("اطلاعات دیتافریم ratings:")
print(df_ratings.head())
print("\nتعداد رتبه‌بندی‌ها:", len(df_ratings))
print("تعداد کاربران منحصر به فرد:", df_ratings["user_id"].nunique())
print("تعداد فیلم‌های منحصر به فرد:", df_ratings["item_id"].nunique())

اطلاعات دیتافریم ratings:
   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596

تعداد رتبه‌بندی‌ها: 100000
تعداد کاربران منحصر به فرد: 943
تعداد فیلم‌های منحصر به فرد: 1682


In [4]:
print("اطلاعات مجموعه ratings:\n")
df_ratings.info()

اطلاعات مجموعه ratings:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   item_id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [5]:
df_ratings.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [ ]:
# ستون‌ها: movie id | movie title | release date | video release date | IMDb URL |
#          unknown | Action | Adventure | Animation | Children's | Comedy | Crime |
#          Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery |
#          Romance | Sci-Fi | Thriller | War | Western
movies_cols = ["item_id", "title", "re_date", "vid_re_date", "imdb_url"] + [
    f"genre_{i}" for i in range(19)
]  # 19 ژانر (از 0 تا 18)
df_movies = pd.read_csv(
    "../data/u.item", sep="|", names=movies_cols, encoding="latin-1"
)

print("\nاطلاعات دیتافریم movies:")
print(df_movies.head(1))
print("\nتعداد فیلم‌ها:", len(df_movies))


اطلاعات دیتافریم movies:
   item_id             title      re_date  vid_re_date                                           imdb_url  genre_0  genre_1  genre_2  genre_3  genre_4  genre_5  genre_6  genre_7  genre_8  genre_9  genre_10  genre_11  genre_12  genre_13  genre_14  genre_15  genre_16  genre_17  genre_18
0        1  Toy Story (1995)  01-Jan-1995          NaN  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0        0        1        1        1        0        0        0        0         0         0         0         0         0         0         0         0         0

تعداد فیلم‌ها: 1682


In [7]:
print("اطلاعات مجموعه ratings:\n")
df_movies.info()

اطلاعات مجموعه ratings:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      1682 non-null   int64  
 1   title        1682 non-null   object 
 2   re_date      1681 non-null   object 
 3   vid_re_date  0 non-null      float64
 4   imdb_url     1679 non-null   object 
 5   genre_0      1682 non-null   int64  
 6   genre_1      1682 non-null   int64  
 7   genre_2      1682 non-null   int64  
 8   genre_3      1682 non-null   int64  
 9   genre_4      1682 non-null   int64  
 10  genre_5      1682 non-null   int64  
 11  genre_6      1682 non-null   int64  
 12  genre_7      1682 non-null   int64  
 13  genre_8      1682 non-null   int64  
 14  genre_9      1682 non-null   int64  
 15  genre_10     1682 non-null   int64  
 16  genre_11     1682 non-null   int64  
 17  genre_12     1682 non-null   int64  
 18  genre_13     1682 non-n

In [8]:
print("داده‌های گمشده:")
df_movies.isnull().sum()

داده‌های گمشده:


item_id           0
title             0
re_date           1
vid_re_date    1682
imdb_url          3
genre_0           0
genre_1           0
genre_2           0
genre_3           0
genre_4           0
genre_5           0
genre_6           0
genre_7           0
genre_8           0
genre_9           0
genre_10          0
genre_11          0
genre_12          0
genre_13          0
genre_14          0
genre_15          0
genre_16          0
genre_17          0
genre_18          0
dtype: int64

In [9]:
df_movies.drop("vid_re_date", axis=1, inplace=True)

In [ ]:
genres = df_movies.columns[5:]
df_movies[genres] = df_movies[genres].astype(int)
df_movies["genres_list"] = df_movies[genres].apply(
    lambda x: [genre for genre, val in zip(genres, x) if val == 1], axis=1
)

print("\nنمونه‌ای از فیلم‌ها با لیست ژانرها:")
print(df_movies[["item_id", "title", "genres_list"]].head())


نمونه‌ای از فیلم‌ها با لیست ژانرها:
   item_id              title                   genres_list
0        1   Toy Story (1995)   [genre_3, genre_4, genre_5]
1        2   GoldenEye (1995)  [genre_1, genre_2, genre_16]
2        3  Four Rooms (1995)                    [genre_16]
3        4  Get Shorty (1995)   [genre_1, genre_5, genre_8]
4        5     Copycat (1995)  [genre_6, genre_8, genre_16]


In [ ]:
df_merged = pd.merge(df_ratings, df_movies, on="item_id")

print("\nاطلاعات دیتافریم ترکیب شده (df_merged):")
print(df_merged.head(1))
print("\nتعداد ردیف‌های دیتافریم ترکیب شده:", len(df_merged))


اطلاعات دیتافریم ترکیب شده (df_merged):
   user_id  item_id  rating  timestamp         title      re_date                                         imdb_url  genre_0  genre_1  genre_2  genre_3  genre_4  genre_5  genre_6  genre_7  genre_8  genre_9  genre_10  genre_11  genre_12  genre_13  genre_14  genre_15  genre_16  genre_17  genre_18 genres_list
0      196      242       3  881250949  Kolya (1996)  24-Jan-1997  http://us.imdb.com/M/title-exact?Kolya%20(1996)        0        0        0        0        0        1        0        0        0        0         0         0         0         0         0         0         0         0         0   [genre_5]

تعداد ردیف‌های دیتافریم ترکیب شده: 100000


In [ ]:
user_movie_matrix = df_merged.pivot_table(
    index="user_id", columns="title", values="rating"
)

print("نمونه‌ای از ماتریس تعامل کاربر-فیلم (User-Item Matrix):")
print(user_movie_matrix.head(1))
print("\nابعاد ماتریس User-Item:", user_movie_matrix.shape)

نمونه‌ای از ماتریس تعامل کاربر-فیلم (User-Item Matrix):
title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  20,000 Leagues Under the Sea (1954)  2001: A Space Odyssey (1968)  3 Ninjas: High Noon At Mega Mountain (1998)  39 Steps, The (1935)  8 1/2 (1963)  8 Heads in a Duffel Bag (1997)  8 Seconds (1994)  A Chef in Love (1996)  Above the Rim (1994)  Absolute Power (1997)  Abyss, The (1989)  Ace Ventura: Pet Detective (1994)  Ace Ventura: When Nature Calls (1995)  Across the Sea of Time (1995)  Addams Family Values (1993)  Addicted to Love (1997)  Addiction, The (1995)  Adventures of Pinocchio, The (1996)  Adventures of Priscilla, Queen of the Desert, The (1994)  Adventures of Robin Hood, The (1938)  Affair to Remember, An (1957)  African Queen, The (1951)  Afterglow (1997)  Age of Innocence, The (1993)  Aiqing wansui (1994)  Air Bud (1997)  Air Force One (1997)  Air Up There, The (1994)  Airheads (1994)  

In [13]:
movie_features = df_movies[genres]

print("نمونه‌ای از ویژگی‌های محتوایی فیلم‌ها (ژانرها):")
print(movie_features.head())

نمونه‌ای از ویژگی‌های محتوایی فیلم‌ها (ژانرها):
   genre_1  genre_2  genre_3  genre_4  genre_5  genre_6  genre_7  genre_8  genre_9  genre_10  genre_11  genre_12  genre_13  genre_14  genre_15  genre_16  genre_17  genre_18
0        0        0        1        1        1        0        0        0        0         0         0         0         0         0         0         0         0         0
1        1        1        0        0        0        0        0        0        0         0         0         0         0         0         0         1         0         0
2        0        0        0        0        0        0        0        0        0         0         0         0         0         0         0         1         0         0
3        1        0        0        0        1        0        0        1        0         0         0         0         0         0         0         0         0         0
4        0        0        0        0        0        1        0        1        0     

In [14]:
temp = []
for item in genres:
    temp.append(item + "_x")

In [ ]:
def create_user_profile(user_id, ratings_df, movie_features_df, min_rating=4):

    user_rated_movies = ratings_df[ratings_df["user_id"] == user_id]
    high_rated_movies = user_rated_movies[user_rated_movies["rating"] >= min_rating]

    if high_rated_movies.empty:
        return pd.Series(0, index=genres)

    high_rated_movie_features = pd.merge(
        high_rated_movies, movie_features_df, on="item_id"
    )
    user_profile = high_rated_movie_features[temp].mean()

    return user_profile

In [ ]:
sample_user_ids = df_ratings["user_id"].unique()[:5]
user_profiles = {}
for user_id in sample_user_ids:
    user_profiles[user_id] = create_user_profile(user_id, df_merged, df_movies)

In [17]:
print("\nنمونه‌ای از پروفایل‌های کاربران:")
for user_id, profile in user_profiles.items():
    print(f"\nپروفایل کاربر {user_id}:")
    print(profile.head())


نمونه‌ای از پروفایل‌های کاربران:

پروفایل کاربر 196:
genre_1_x    0.000000
genre_2_x    0.045455
genre_3_x    0.000000
genre_4_x    0.045455
genre_5_x    0.818182
dtype: float64

پروفایل کاربر 186:
genre_1_x    0.416667
genre_2_x    0.145833
genre_3_x    0.062500
genre_4_x    0.104167
genre_5_x    0.104167
dtype: float64

پروفایل کاربر 22:
genre_1_x    0.621622
genre_2_x    0.310811
genre_3_x    0.000000
genre_4_x    0.000000
genre_5_x    0.472973
dtype: float64

پروفایل کاربر 244:
genre_1_x    0.136364
genre_2_x    0.064935
genre_3_x    0.051948
genre_4_x    0.045455
genre_5_x    0.402597
dtype: float64

پروفایل کاربر 166:
genre_1_x    0.545455
genre_2_x    0.000000
genre_3_x    0.000000
genre_4_x    0.090909
genre_5_x    0.181818
dtype: float64


In [ ]:
movie_features_indexed = df_movies.set_index("item_id")[genres]
movie_similarity_matrix = cosine_similarity(movie_features_indexed)

movie_similarity_df = pd.DataFrame(
    movie_similarity_matrix, index=df_movies["item_id"], columns=df_movies["item_id"]
)

print("\nنمونه‌ای از ماتریس شباهت فیلم‌ها:")
print(movie_similarity_df.iloc[:5, :5])


نمونه‌ای از ماتریس شباهت فیلم‌ها:
item_id         1         2        3         4         5
item_id                                                 
1        1.000000  0.000000  0.00000  0.333333  0.000000
2        0.000000  1.000000  0.57735  0.333333  0.333333
3        0.000000  0.577350  1.00000  0.000000  0.577350
4        0.333333  0.333333  0.00000  1.000000  0.333333
5        0.000000  0.333333  0.57735  0.333333  1.000000


In [ ]:
def get_content_based_recommendations(user_id, num_recommendations=10):
    user_profile = create_user_profile(user_id, df_merged, df_movies)

    # فیلم‌هایی که کاربر قبلاً تماشا کرده است
    watched_movies_ids = df_merged[df_merged["user_id"] == user_id]["item_id"].tolist()

    # فیلم‌های ندیده (که کاربر هنوز رتبه‌بندی نکرده است)
    unwatched_movies_df = df_movies[~df_movies["item_id"].isin(watched_movies_ids)]
    unwatched_movie_features = unwatched_movies_df.set_index("item_id")[genres]

    if unwatched_movie_features.empty or user_profile.isnull().all():
        return pd.DataFrame(columns=["title", "content_score"])

    # محاسبه شباهت بین پروفایل کاربر و فیلم‌های ندیده
    # (پروفایل کاربر یک بردار 1xN و ویژگی‌های فیلم‌ها یک ماتریس MxN هستند)
    # نتیجه یک بردار Mx1 خواهد بود که شباهت هر فیلم به پروفایل کاربر را نشان می‌دهد.
    content_scores = cosine_similarity(
        user_profile.values.reshape(1, -1), unwatched_movie_features
    )

    # تبدیل به سری pandas برای سهولت در مرتب‌سازی
    content_scores_series = pd.Series(
        content_scores.flatten(), index=unwatched_movie_features.index
    )

    # مرتب‌سازی فیلم‌ها بر اساس امتیاز محتوا و انتخاب بهترین‌ها
    top_recommendations = content_scores_series.sort_values(ascending=False).head(
        num_recommendations
    )

    # اضافه کردن عنوان فیلم‌ها
    recommended_movies_info = df_movies[
        df_movies["item_id"].isin(top_recommendations.index)
    ][["item_id", "title"]]
    recommended_movies_info = recommended_movies_info.set_index("item_id")
    recommended_movies_info["content_score"] = top_recommendations

    return recommended_movies_info.sort_values(by="content_score", ascending=False)

In [20]:
for i in range(5):
    user_id_to_test = np.random.randint(943)
    content_based_recs = get_content_based_recommendations(user_id_to_test)
    print(f"\nپیشنهادهای محتوا-محور برای کاربر {user_id_to_test}:")
    print(content_based_recs)
    print("--------------")


پیشنهادهای محتوا-محور برای کاربر 561:
                                            title  content_score
item_id                                                         
337                      House of Yes, The (1997)       0.860143
150                               Swingers (1996)       0.820390
316                     As Good As It Gets (1997)       0.820390
522                            Down by Law (1986)       0.820390
856                         Night on Earth (1991)       0.820390
1165                             Big Bully (1996)       0.820390
1186                          Inkwell, The (1994)       0.820390
1312                 Pompatus of Love, The (1996)       0.820390
1456                        Beat the Devil (1954)       0.820390
1569     Vie est belle, La (Life is Rosey) (1987)       0.820390
--------------

پیشنهادهای محتوا-محور برای کاربر 3:
                                      title  content_score
item_id                                                   
914        

In [ ]:
# نرمال‌سازی ماتریس User-Item (میانگین رتبه‌بندی کاربر از رتبه‌ها کسر می‌شود)
user_movie_matrix_normalized = user_movie_matrix.apply(lambda x: x - x.mean(), axis=1)

# پر کردن مقادیر NaN با صفر برای محاسبه شباهت (این مقادیر در محاسبه شباهت نادیده گرفته می‌شوند)
user_movie_matrix_normalized_filled = user_movie_matrix_normalized.fillna(0)

# محاسبه ماتریس شباهت کاربران با استفاده از cosine similarity
user_similarity_matrix = cosine_similarity(user_movie_matrix_normalized_filled)

# تبدیل ماتریس شباهت به DataFrame برای خوانایی بیشتر
user_similarity_df = pd.DataFrame(
    user_similarity_matrix,
    index=user_movie_matrix.index,
    columns=user_movie_matrix.index,
)

print("\nنمونه‌ای از ماتریس شباهت کاربران:")
print(user_similarity_df.iloc[:5, :5])


نمونه‌ای از ماتریس شباهت کاربران:
user_id         1         2         3         4         5
user_id                                                  
1        1.000000  0.044104  0.010664  0.059305  0.135339
2        0.044104  1.000000  0.013084 -0.016534  0.036012
3        0.010664  0.013084  1.000000 -0.059226  0.016495
4        0.059305 -0.016534 -0.059226  1.000000  0.007373
5        0.135339  0.036012  0.016495  0.007373  1.000000


In [ ]:
def predict_collaborative_rating(
    user_id, movie_title, user_movie_matrix, user_similarity_df, k=20
):
    if movie_title not in user_movie_matrix.columns:
        return 0  # فیلم در ماتریس وجود ندارد

    # پیدا کردن k نزدیک‌ترین کاربران به user_id
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)
    similar_users = similar_users.drop(user_id)  # کاربر خودش را حذف می‌کنیم

    # انتخاب k کاربر برتر (همسایه‌های نزدیک)
    top_k_similar_users = similar_users.head(k)

    numerator = 0
    denominator = 0

    # میانگین رتبه‌بندی کاربر هدف
    user_mean_rating = user_movie_matrix.loc[user_id].mean()

    for sim_user_id, similarity_score in top_k_similar_users.items():
        # بررسی اینکه آیا کاربر مشابه فیلم مورد نظر را رتبه‌بندی کرده است
        if not pd.isna(user_movie_matrix.loc[sim_user_id, movie_title]):
            sim_user_rating = user_movie_matrix.loc[sim_user_id, movie_title]
            sim_user_mean_rating = user_movie_matrix.loc[sim_user_id].mean()

            numerator += similarity_score * (sim_user_rating - sim_user_mean_rating)
            denominator += abs(similarity_score)

    if denominator == 0:
        return user_mean_rating  # اگر هیچ کاربر مشابهی فیلم را رتبه‌بندی نکرده بود، میانگین رتبه‌های کاربر هدف را برمی‌گردانیم

    predicted_rating = user_mean_rating + (numerator / denominator)

    # اطمینان از اینکه رتبه در بازه 1 تا 5 قرار دارد
    return max(1, min(5, predicted_rating))

In [ ]:
for i in range(5):
    user_id_test_cf = np.random.randint(943)
    movie_title_test_cf = "Toy Story (1995)"
    predicted_rating_cf = predict_collaborative_rating(
        user_id_test_cf, movie_title_test_cf, user_movie_matrix, user_similarity_df
    )
    print(
        f"\nرتبه پیش‌بینی شده مشارکتی برای کاربر {user_id_test_cf} و فیلم '{movie_title_test_cf}': {predicted_rating_cf:.2f}"
    )


رتبه پیش‌بینی شده مشارکتی برای کاربر 419 و فیلم 'Toy Story (1995)': 3.85

رتبه پیش‌بینی شده مشارکتی برای کاربر 401 و فیلم 'Toy Story (1995)': 3.00

رتبه پیش‌بینی شده مشارکتی برای کاربر 829 و فیلم 'Toy Story (1995)': 3.67

رتبه پیش‌بینی شده مشارکتی برای کاربر 69 و فیلم 'Toy Story (1995)': 4.00

رتبه پیش‌بینی شده مشارکتی برای کاربر 793 و فیلم 'Toy Story (1995)': 4.09


In [ ]:
def get_collaborative_recommendations(user_id, num_recommendations=10):
    # فیلم‌هایی که کاربر قبلاً تماشا کرده است
    watched_movies = df_merged[df_merged["user_id"] == user_id]["title"].tolist()

    # فیلم‌های ندیده
    all_movies_titles = user_movie_matrix.columns.tolist()
    unwatched_movies_titles = [
        movie for movie in all_movies_titles if movie not in watched_movies
    ]

    if not unwatched_movies_titles:
        return pd.DataFrame(columns=["title", "collaborative_score"])

    collaborative_scores = {}
    for movie_title in unwatched_movies_titles:
        score = predict_collaborative_rating(
            user_id, movie_title, user_movie_matrix, user_similarity_df
        )
        collaborative_scores[movie_title] = score

    # تبدیل به DataFrame و مرتب‌سازی
    collaborative_recs_df = pd.DataFrame(
        list(collaborative_scores.items()), columns=["title", "collaborative_score"]
    )

    return collaborative_recs_df.sort_values(
        by="collaborative_score", ascending=False
    ).head(num_recommendations)

In [25]:
for i in range(5):
    user_id_to_test_cf_recs = np.random.randint(943)
    collaborative_recs = get_collaborative_recommendations(user_id_to_test_cf_recs)
    print(f"\nپیشنهادهای مشارکتی برای کاربر {user_id_to_test_cf_recs}:")
    print(collaborative_recs)


پیشنهادهای مشارکتی برای کاربر 76:
                                                  title  collaborative_score
1409                         Thin Blue Line, The (1988)             5.000000
1571                         Wrong Trousers, The (1993)             5.000000
1279                                     Sleeper (1973)             5.000000
949                              Misérables, Les (1995)             5.000000
305                               Close Shave, A (1995)             5.000000
1519  Wallace & Gromit: The Best of Aardman Animatio...             5.000000
1089                             Pather Panchali (1955)             4.957015
555                                     Gaslight (1944)             4.957015
395                            Dial M for Murder (1954)             4.957015
81                                     Aparajito (1956)             4.957015

پیشنهادهای مشارکتی برای کاربر 722:
                             title  collaborative_score
221             Brassed Of

In [ ]:
def get_hybrid_recommendations(user_id, alpha=0.5, num_recommendations=10):
    """
    این تابع پیشنهادهای ترکیبی را برای یک کاربر خاص تولید می‌کند.

    Parameters:
    user_id (int): شناسه‌ی کاربر.
    alpha (float): وزن برای فیلترینگ مشارکتی (مقدار بین 0 و 1).
                   1-alpha وزن برای فیلترینگ محتوا-محور است.
    num_recommendations (int): تعداد پیشنهادهای مورد نیاز.
    """

    # 1. دریافت امتیازات محتوا-محور
    watched_movies_ids = df_merged[df_merged["user_id"] == user_id]["item_id"].tolist()
    unwatched_movies_df = df_movies[~df_movies["item_id"].isin(watched_movies_ids)]
    unwatched_movie_features = unwatched_movies_df.set_index("item_id")[genres]

    if unwatched_movie_features.empty:
        return pd.DataFrame(columns=["title", "final_score"])

    user_profile = create_user_profile(user_id, df_merged, df_movies)

    if user_profile.isnull().all():
        content_scores_series = pd.Series(0, index=unwatched_movie_features.index)
    else:
        content_scores = cosine_similarity(
            user_profile.values.reshape(1, -1), unwatched_movie_features
        )
        content_scores_series = pd.Series(
            content_scores.flatten(), index=unwatched_movie_features.index
        )

    # نرمال‌سازی امتیازات محتوا-محور به بازه 1 تا 5
    min_score = content_scores_series.min()
    max_score = content_scores_series.max()
    if (max_score - min_score) != 0:
        content_scores_normalized = 1 + 4 * (content_scores_series - min_score) / (
            max_score - min_score
        )
    else:
        content_scores_normalized = pd.Series(3, index=content_scores_series.index)

    # 2. دریافت امتیازات مشارکتی
    collaborative_scores = {}

    # اینجا تغییر اعمال می‌شود:
    # به جای استفاده از title، مستقیماً از item_id فیلم‌های ندیده استفاده می‌کنیم
    unwatched_movie_ids = unwatched_movie_features.index.tolist()

    for item_id in unwatched_movie_ids:
        movie_title = df_movies[df_movies["item_id"] == item_id]["title"].iloc[
            0
        ]  # گرفتن عنوان فیلم از item_id
        collab_score = predict_collaborative_rating(
            user_id, movie_title, user_movie_matrix, user_similarity_df
        )
        collaborative_scores[item_id] = collab_score  # ذخیره با item_id به عنوان کلید

    collaborative_scores_series = pd.Series(collaborative_scores)

    # اکنون collaborative_scores_series مستقیماً با item_idها ایندکس شده است، نیازی به نگاشت عنوان به item_id نیست.

    # اطمینان از اینکه هر دو سری ایندکس‌های یکسانی دارند (فقط فیلم‌های ندیده)
    common_indices = unwatched_movie_features.index.intersection(
        collaborative_scores_series.index
    )

    content_scores_final = content_scores_normalized[common_indices]
    collaborative_scores_final = collaborative_scores_series[common_indices]

    # 3. ترکیب امتیازات
    final_scores = (
        alpha * collaborative_scores_final + (1 - alpha) * content_scores_final
    )

    # تبدیل به DataFrame و مرتب‌سازی
    recommended_movies_info = df_movies[df_movies["item_id"].isin(final_scores.index)][
        ["item_id", "title"]
    ]
    recommended_movies_info = recommended_movies_info.set_index("item_id")
    recommended_movies_info["final_score"] = final_scores

    return recommended_movies_info.sort_values(by="final_score", ascending=False).head(
        num_recommendations
    )

In [27]:
# تست برای کاربر 1 با آلفای 0.5
user_id_to_test_hybrid = 1
hybrid_recs_alpha_0_5 = get_hybrid_recommendations(user_id_to_test_hybrid, alpha=0.5)
print(f"\nپیشنهادهای ترکیبی برای کاربر {user_id_to_test_hybrid} با آلفا = 0.5:")
print(hybrid_recs_alpha_0_5)


پیشنهادهای ترکیبی برای کاربر 1 با آلفا = 0.5:
                                               title  final_score
item_id                                                          
517                                 Manhattan (1979)     5.000000
856                            Night on Earth (1991)     4.981434
522                               Down by Law (1986)     4.899834
512                           Wings of Desire (1987)     4.869615
316                        As Good As It Gets (1997)     4.760626
317                 In the Name of the Father (1993)     4.721019
851      Two or Three Things I Know About Her (1966)     4.721019
1021                                    8 1/2 (1963)     4.721019
909                          Dangerous Beauty (1998)     4.721019
868                          Hearts and Minds (1996)     4.721019


In [ ]:
# تست برای کاربر 1 با آلفای 0.1 (بیشتر محتوا-محور)
hybrid_recs_alpha_0_1 = get_hybrid_recommendations(user_id_to_test_hybrid, alpha=0.1)
print(
    f"\nپیشنهادهای ترکیبی برای کاربر {user_id_to_test_hybrid} با آلفا = 0.1 (بیشتر محتوا-محور):"
)
print(hybrid_recs_alpha_0_1)


پیشنهادهای ترکیبی برای کاربر 1 با آلفا = 0.1 (بیشتر محتوا-محور):
                               title  final_score
item_id                                          
517                 Manhattan (1979)     5.000000
512           Wings of Desire (1987)     4.973923
856            Night on Earth (1991)     4.966582
522               Down by Law (1986)     4.950262
316        As Good As It Gets (1997)     4.922420
731          Corrina, Corrina (1994)     4.918599
347               Wag the Dog (1997)     4.902376
936               Brassed Off (1996)     4.899398
1475       Bhaji on the Beach (1993)     4.899148
789      Swimming with Sharks (1995)     4.887580


In [ ]:
# تست برای کاربر 1 با آلفای 0.9 (بیشتر مشارکتی)
hybrid_recs_alpha_0_9 = get_hybrid_recommendations(user_id_to_test_hybrid, alpha=0.9)
print(
    f"\nپیشنهادهای ترکیبی برای کاربر {user_id_to_test_hybrid} با آلفا = 0.9 (بیشتر مشارکتی):"
)
print(hybrid_recs_alpha_0_9["final_score"].values)


پیشنهادهای ترکیبی برای کاربر 1 با آلفا = 0.9 (بیشتر مشارکتی):
[5.         4.99628688 4.94420373 4.94420373 4.94420373 4.94420373
 4.94420373 4.94420373 4.94420373 4.94420373]


In [ ]:
temp_dic = {
    "movie_0_1": hybrid_recs_alpha_0_1["title"].values,
    "score_0_1": hybrid_recs_alpha_0_1["final_score"].values,
    "movie_0_5": hybrid_recs_alpha_0_5["title"].values,
    "score_0_5": hybrid_recs_alpha_0_5["final_score"].values,
    "movie_0_9": hybrid_recs_alpha_0_9["title"].values,
    "score_0_9": hybrid_recs_alpha_0_9["final_score"].values,
}
submission = pd.DataFrame(temp_dic)
submission

,movie_0_1,score_0_1,movie_0_5,score_0_5,movie_0_9,score_0_9
0,Manhattan (1979),5.000000,Manhattan (1979),5.000000,Manhattan (1979),5.000000
1,Wings of Desire (1987),4.973923,Night on Earth (1991),4.981434,Night on Earth (1991),4.996287
2,Night on Earth (1991),4.966582,Down by Law (1986),4.899834,Career Girls (1997),4.944204
3,Down by Law (1986),4.950262,Wings of Desire (1987),4.869615,In the Name of the Father (1993),4.944204
4,As Good As It Gets (1997),4.922420,As Good As It Gets (1997),4.760626,8 1/2 (1963),4.944204
5,"Corrina, Corrina (1994)",4.918599,In the Name of the Father (1993),4.721019,Flirt (1995),4.944204
6,Wag the Dog (1997),4.902376,Two or Three Things I Know About Her (1966),4.721019,Two or Three Things I Know About Her (1966),4.944204
7,Brassed Off (1996),4.899398,8 1/2 (1963),4.721019,East of Eden (1955),4.944204
8,Bhaji on the Beach (1993),4.899148,Dangerous Beauty (1998),4.721019,Once Upon a Time... When We Were Colored (1995),4.944204
9,Swimming with Sharks (1995),4.887580,Hearts and Minds (1996),4.721019,Before the Rain (Pred dozhdot) (1994),4.944204


In [ ]:
# تقسیم داده‌ها به مجموعه آموزش و آزمون
train_data, test_data = train_test_split(df_merged, test_size=0.2, random_state=42)

print("\nابعاد مجموعه آموزش:", train_data.shape)
print("ابعاد مجموعه آزمون:", test_data.shape)

# ساخت ماتریس تعامل کاربر - فیلم برای مجموعه آموزش
train_user_movie_matrix = train_data.pivot_table(
    index="user_id", columns="title", values="rating"
)
train_user_movie_matrix_normalized = train_user_movie_matrix.apply(
    lambda x: x - x.mean(), axis=1
)
train_user_movie_matrix_normalized_filled = train_user_movie_matrix_normalized.fillna(0)

# محاسبه ماتریس شباهت کاربران برای مجموعه آموزش
train_user_similarity_matrix = cosine_similarity(
    train_user_movie_matrix_normalized_filled
)
train_user_similarity_df = pd.DataFrame(
    train_user_similarity_matrix,
    index=train_user_movie_matrix.index,
    columns=train_user_movie_matrix.index,
)

print("\nابعاد ماتریس User-Item آموزش:", train_user_movie_matrix.shape)
print("ابعاد ماتریس شباهت کاربران آموزش:", train_user_similarity_df.shape)


ابعاد مجموعه آموزش: (80000, 27)
ابعاد مجموعه آزمون: (20000, 27)

ابعاد ماتریس User-Item آموزش: (943, 1635)
ابعاد ماتریس شباهت کاربران آموزش: (943, 943)


In [ ]:
def evaluate_recommender(recommender_func, test_df, k=10, min_rating_threshold=4):
    all_users = test_df["user_id"].unique()
    precisions = []

    for user_id in all_users:
        user_test_ratings = test_df[test_df["user_id"] == user_id]
        relevant_items = user_test_ratings[
            user_test_ratings["rating"] >= min_rating_threshold
        ]["item_id"].tolist()

        if not relevant_items:
            continue

        recommended_items_df = recommender_func(user_id, num_recommendations=k)
        recommended_items = recommended_items_df.index.tolist()

        hits = 0
        for item_id in recommended_items:
            if item_id in relevant_items:
                hits += 1

        if len(recommended_items) > 0:
            precision_at_k = hits / len(recommended_items)
            precisions.append(precision_at_k)

    if precisions:
        return np.mean(precisions)
    else:
        return 0.0


def hybrid_recommender_for_eval(user_id, num_recommendations):
    return get_hybrid_recommendations(
        user_id, alpha=0.5, num_recommendations=num_recommendations
    )


mean_precision = evaluate_recommender(hybrid_recommender_for_eval, test_data, k=10)
print(f"\nمیانگین Precision@10 برای سیستم ترکیبی: {mean_precision:.4f}")

alphas_to_test = [0.1, 0.3, 0.5, 0.7, 0.9]
precision_results = []

for alpha_val in alphas_to_test:

    def recommender_alpha_eval(user_id, num_recommendations):
        return get_hybrid_recommendations(
            user_id, alpha=alpha_val, num_recommendations=num_recommendations
        )

    precision_val = evaluate_recommender(recommender_alpha_eval, test_data, k=10)
    precision_results.append(precision_val)
    print(f"Precision@10 برای آلفا = {alpha_val}: {precision_val:.4f}")


میانگین Precision@10 برای سیستم ترکیبی: 0.0000
Precision@10 برای آلفا = 0.1: 0.0000
